In [1]:
#load data

import pandas as pd
from loaddata import defineTestSet,defineResponse,defineFeatures,defineTrainingSets,defineSplits
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc,roc_auc_score
import pickle
import seaborn as sns
from datetime import datetime
import os
from config import ensemble_dir, rand_var,z_score_ensemble_dir
from tools.make_model import optimise_logres_featsel,optimise_rf_featsel,optimise_svc_featsel
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import RandomizedSearchCV

In [2]:
her2=0
rcut = 1
feats_chemo = defineFeatures('chemo', her2=her2)
feats_clinical = defineFeatures('clinical', her2=her2)

In [3]:
#traing data



df_train = pd.read_csv('inputs/training_df.csv')

chemo_Xtrain, chemo_ytrainCateg, chemo_ytrainScore, chemo_ytrainID = defineTrainingSets(df_train, feats_chemo, her2=her2)
chemo_splits = defineSplits(chemo_Xtrain, chemo_ytrainCateg)


clinical_Xtrain, clinical_ytrainCateg, clinical_ytrainScore, clinical_ytrainID = defineTrainingSets(df_train, feats_clinical, her2=her2)
clinical_splits = defineSplits(clinical_Xtrain, clinical_ytrainCateg)



ytrain_pCR = defineResponse(df_train, 'pCR', her2=her2)

In [4]:
#load test data

df_test_pCR_pos = pd.read_csv('inputs//testing_her2pos_df.csv')
df_test_pCR_neg = pd.read_csv('inputs//testing_her2neg_df.csv')




chemo_x_test_pCR_pos = defineTestSet(df_test_pCR_pos,feats_chemo,her2=her2)
chemo_x_test_pCR_neg = defineTestSet(df_test_pCR_neg,feats_chemo,her2=her2)
chemo_x_test_comb = pd.concat([chemo_x_test_pCR_pos,chemo_x_test_pCR_neg])

clinical_x_test_pCR_pos = defineTestSet(df_test_pCR_pos,feats_clinical,her2=her2)
clinical_x_test_pCR_neg = defineTestSet(df_test_pCR_neg,feats_clinical,her2=her2)
clinical_x_test_comb = pd.concat([clinical_x_test_pCR_pos,clinical_x_test_pCR_neg])




y_test_pCR_pos = defineResponse(df_test_pCR_pos, 'pCR', her2=her2)
y_test_pCR_neg = defineResponse(df_test_pCR_neg, 'pCR', her2=her2)

y_test_comb = pd.concat([y_test_pCR_pos,y_test_pCR_neg])

## Fig 4b z score calculations

In [ ]:


base_logres_model = optimise_logres_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_
base_rf_model = optimise_rf_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_
base_svc_model = optimise_svc_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_




used_model_list = [
    ('Logistic Regression',base_logres_model),
    ('Random Forest Classifier',base_rf_model),
    ('Support Vector Classifier',base_svc_model)

    
    
]



In [ ]:
min_weight =0
max_weight = 5

weights_of_models = range(min_weight,max_weight,1)

possible_combinations = [
    [w1,w2,w3]
    for w1 in weights_of_models
    for w2 in weights_of_models
    for w3 in weights_of_models

]

filtered_combinations = [i for i in possible_combinations if any(i)]#remove all 0 combination which will break voting classifier


param_grid = {
 'weights':filtered_combinations   
}

base_ensemble_model_weighted = VotingClassifier(estimators=used_model_list, voting='soft')

base_ensemble_search = RandomizedSearchCV(base_ensemble_model_weighted, param_grid, cv=chemo_splits,scoring='roc_auc',return_train_score=True, n_jobs=-1, verbose=0,n_iter=1500,random_state=rand_var)

base_ensemble_search.fit(chemo_Xtrain,ytrain_pCR)




In [ ]:

best_base_ensemble_model = base_ensemble_search.best_estimator_

y_pred_prob_base = best_base_ensemble_model.predict_proba(chemo_x_test_comb)[:, 1]

base_auc = roc_auc_score(y_test_comb, y_pred_prob_base)



In [ ]:
#dataframe to save the z scores

feature_importance = pd.DataFrame(index=chemo_x_test_pCR_pos.columns, columns=['z-score'])

In [ ]:

for feature in chemo_x_test_pCR_pos.columns:
    x_without_feat = chemo_Xtrain.drop(columns=[feature])
    
    without_feat_logres_model = optimise_logres_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_
    without_feat_rf_model = optimise_rf_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_
    without_feat_svc_model = optimise_svc_featsel(chemo_Xtrain,ytrain_pCR,cut=float(rcut),cv=chemo_splits,max = 2000).best_estimator_




    without_feat_used_model_list = [
        ('Logistic Regression',without_feat_logres_model),
        ('Random Forest Classifier',without_feat_rf_model),
        ('Support Vector Classifier',without_feat_svc_model)

        
        
    ]
    
    
    without_ensemble_model_weighted = VotingClassifier(estimators=without_feat_used_model_list, voting='soft')

    without_ensemble_search = RandomizedSearchCV(base_ensemble_model_weighted, param_grid, cv=chemo_splits,scoring='roc_auc',return_train_score=True, n_jobs=-1, verbose=0,n_iter=1500,random_state=rand_var)

    without_ensemble_search.fit(x_without_feat,ytrain_pCR)
    
    without_ensemble_best_model = without_ensemble_search.best_estimator_
    
    test_x_without_feat = chemo_x_test_comb.drop(columns=[feature])
    

    y_without_feat_pred_prob = without_ensemble_best_model.predict_proba(test_x_without_feat)[:, 1]
    
    auc_without_feat = roc_auc_score(y_test_comb, y_without_feat_pred_prob)
    
    z_score = (auc_without_feat - base_auc) / np.std(y_pred_prob_base)
    
    feature_importance.loc[feature, 'z-score'] = z_score

In [ ]:
feature_importance['z-score'] = pd.to_numeric(feature_importance['z-score'], errors='coerce')

In [ ]:
os.makedirs(z_score_ensemble_dir, exist_ok=True)

tm = datetime.now()
modelname = 'LRRFSVC'
filename = "EnsembleZscore_model_{}_random_{}_Date_{}_{}_{}_{}.p".format(modelname,rand_var,tm.year,tm.month,tm.day,tm.strftime("%H_%M_%S"))


with open(z_score_ensemble_dir+filename,'wb') as w:
    pickle.dump(feature_importance,w)

## Fig 4b graphing

In [ ]:
with open(z_score_ensemble_dir+"EnsembleZscore_model_LRRFSVC_random_123_Date_2023_7_27_06_04_54.p",'rb') as w:
       
    feature_df = pickle.load(w, encoding='utf-8')

In [ ]:
fig, ax = plt.subplots(figsize=(1, 10))

sns.heatmap(feature_df, cmap='coolwarm', annot=False, fmt=".3f", ax=ax,linewidths=3)


ax.set_xlabel("Models")
ax.set_ylabel("Features")
ax.set_title("Z-score Heatmap")

plt.savefig('images/EDfig9b', bbox_inches='tight', transparent=False, dpi=300)

plt.show()

## Figure 4c

In [ ]:
#ensemble models to use

with open(ensemble_dir+"Modelnum_3_random_123_Feats_clinical_Date_2023_7_26_15_39_12.p",'rb') as w:
       
    clinical_ensemble = pickle.load(w, encoding='utf-8')
    
    
with open(ensemble_dir+"Modelnum_3_random_123_Feats_chemo_Date_2023_7_26_15_34_25.p",'rb') as w:
       
    chemo_ensemble = pickle.load(w, encoding='utf-8')



In [ ]:
clinical_y_pred = clinical_ensemble.predict(clinical_x_test_comb)
chemo_y_pred = chemo_ensemble.predict(chemo_x_test_comb)

  

In [ ]:



plt.figure()
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    

    

fp_rate, tp_rate, thresholds = roc_curve(y_test_comb, clinical_y_pred)
        
roc_auc = auc(fp_rate, tp_rate)
   
plt.plot(fp_rate, tp_rate, color='black', lw=2, linestyle='--',label='Clinical (Validation AUC = %0.2f)' % roc_auc)
    
    
    
    
fp_rate, tp_rate, thresholds = roc_curve(y_test_comb, chemo_y_pred)
        
roc_auc = auc(fp_rate, tp_rate)
   
plt.plot(fp_rate, tp_rate, color='black', lw=2, label='Chemo (Validation AUC = %0.2f)' % roc_auc)
    

    
    
plt.legend(loc='lower right')
    
plt.show()


## Extended Figure 9g

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
with open(ensemble_dir+"Modelnum_3_random_123_Feats_clinical_Date_2023_7_26_15_39_12.p",'rb') as w:
       
    clinical_ensemble = pickle.load(w, encoding='utf-8')
    
    
with open(ensemble_dir+"Modelnum_3_random_123_Feats_chemo_Date_2023_7_26_15_34_25.p",'rb') as w:
       
    chemo_ensemble = pickle.load(w, encoding='utf-8')

In [ ]:
clinical_predictions = clinical_ensemble.predict_proba(clinical_x_test_comb)[:, 1]
chemo_predictions = chemo_ensemble.predict_proba(chemo_x_test_comb)[:, 1]

In [ ]:
clinical_precision, clinical_recall, clinical_thresholds = precision_recall_curve(y_test_comb, clinical_predictions)
chemo_precision, chemo_recall, chemo_thresholds = precision_recall_curve(y_test_comb, chemo_predictions)

In [ ]:
plt.xlabel('Recall')
plt.ylabel('Precision')



plt.plot(clinical_recall, clinical_precision, color='red', lw=2, label='Clinical model')
plt.fill_between(clinical_recall, clinical_precision, alpha=0.2, color='red')

plt.plot(chemo_recall, chemo_precision, color='blue', lw=2, label='Chemo model')
plt.fill_between(chemo_recall, chemo_precision, alpha=0.2, color='blue')

plt.axhline(y=np.mean(y_test_comb), color='black', linestyle='--', label='Random Performance')


plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
plt.savefig("./images/extended_figure_9g.png", bbox_inches='tight', transparent=False, dpi=300)

## Figure 4D

In [5]:
with open(ensemble_dir+"Modelnum_3_random_123_Feats_clinical_Date_2023_7_26_15_39_12.p",'rb') as w:
       
    clinical_ensemble = pickle.load(w, encoding='utf-8')
    
    
with open(ensemble_dir+"Modelnum_3_random_123_Feats_chemo_Date_2023_7_26_15_34_25.p",'rb') as w:
       
    chemo_ensemble = pickle.load(w, encoding='utf-8')

In [6]:
train_auc_clinical = []
train_auc_chemo = []


In [7]:
for data in chemo_splits:
    train_id,test_id = data[0],data[1]
    
    y_expected = ytrain_pCR.loc[test_id]
    
    clinical_x = clinical_Xtrain.loc[test_id]
    chemo_x = chemo_Xtrain.loc[test_id]
    
    clinical_pred = clinical_ensemble.predict_proba(clinical_x)[:, 1]
    chemo_pred = chemo_ensemble.predict_proba(chemo_x)[:, 1]
    
    
    
    fp_rate, tp_rate, thresholds = roc_curve(y_expected, clinical_pred)
        
    roc_auc = auc(fp_rate, tp_rate)
    
    train_auc_clinical.append(roc_auc)
    
    
    
    
    fp_rate, tp_rate, thresholds = roc_curve(y_expected, chemo_pred)
        
    roc_auc = auc(fp_rate, tp_rate)
    
    train_auc_chemo.append(roc_auc)
    

    

In [10]:
clinical_pred = clinical_ensemble.predict_proba(clinical_x_test_comb)[:, 1]
chemo_pred = chemo_ensemble.predict_proba(chemo_x_test_comb)[:, 1]

In [11]:
clinical_fp_rate, clinical_tp_rate, thresholds = roc_curve(y_test_comb, clinical_pred)
chemo_fp_rate, chemo_tp_rate, thresholds = roc_curve(y_test_comb, chemo_pred)

In [12]:
clinical_roc_auc = auc(clinical_fp_rate, clinical_tp_rate)
chemo_roc_auc = auc(chemo_fp_rate, chemo_tp_rate)